In [25]:
#james chartouni
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, RobustScaler

In [3]:
training_data = pd.read_csv("cleaned_input/train_consolidated.csv")
training_data = training_data.drop(['msno'], axis=1)

In [4]:
#split data 
y = training_data["is_churn"].values
X = training_data.drop(["is_churn"], axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15, shuffle=True)#change to stratified shuffle split

In [30]:
training_data.


,Unnamed: 0,is_churn,trans_count,logs_count,Unnamed: 0.1,bd,registration_init_year,registration_init_month,registration_init_date,expiration_date_year,...,long_time_user,gender_encoded0,gender_encoded1,gender_encoded2,city_encoded,gender_encoded,registered_via_encoded,payment_method_id_encoded,is_auto_renew_encoded,is_cancel_encoded
0,0,1,2.0,26.0,4396024.0,36.0,2005.0,4.0,6.0,2017.0,...,1,1,0,0,17,0,4,32,1,1
1,1,1,23.0,521.0,1941053.0,38.0,2005.0,4.0,7.0,2017.0,...,1,0,1,0,9,1,4,33,2,2
2,2,1,10.0,237.0,1587455.0,27.0,2005.0,10.0,16.0,2017.0,...,1,1,0,0,10,0,4,33,2,2
3,3,1,2.0,735.0,1500627.0,23.0,2005.0,11.0,2.0,2017.0,...,1,1,0,0,12,0,4,32,1,1
4,4,1,8.0,758.0,2600269.0,27.0,2005.0,12.0,28.0,2017.0,...,1,0,1,0,2,1,4,32,1,1


In [31]:
#logistic regression 

logistic_pipe = make_pipeline(LogisticRegression())
score = cross_val_score(logistic_pipe, X_train, y_train, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))



ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [9]:
#Light GBM 
# create dataset for lightgbm
# if you want to re-use data, remember to set free_raw_data=False
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train,free_raw_data=False)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# generate a feature name
feature_name = X_train.dtype.names

print('Start training...')
# feature_name and categorical_feature
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=50,
                valid_sets=lgb_train,  # eval training data
              )

# save model to file
gbm.save_model('model.txt')

# feature names
print('Feature names:', gbm.feature_name())

# feature importances
print('Feature importances:', list(gbm.feature_importance()))


Start training...
[1]	training's binary_logloss: 0.644414
[2]	training's binary_logloss: 0.600319
[3]	training's binary_logloss: 0.560226
[4]	training's binary_logloss: 0.523623
[5]	training's binary_logloss: 0.490088
[6]	training's binary_logloss: 0.459267
[7]	training's binary_logloss: 0.433117
[8]	training's binary_logloss: 0.408953
[9]	training's binary_logloss: 0.384325
[10]	training's binary_logloss: 0.361483
[11]	training's binary_logloss: 0.340258
[12]	training's binary_logloss: 0.320505
[13]	training's binary_logloss: 0.302091
[14]	training's binary_logloss: 0.284904
[15]	training's binary_logloss: 0.26884
[16]	training's binary_logloss: 0.253811
[17]	training's binary_logloss: 0.239734
[18]	training's binary_logloss: 0.226533
[19]	training's binary_logloss: 0.214146
[20]	training's binary_logloss: 0.202512
[21]	training's binary_logloss: 0.191576
[22]	training's binary_logloss: 0.181289
[23]	training's binary_logloss: 0.171607
[24]	training's binary_logloss: 0.162487
[25]	tra